# **Template OP on salish**

In [1]:
%matplotlib inline

import sys
import xarray as xr
import numpy as np
import os
import math
from datetime import datetime, timedelta
from parcels import FieldSet, Field, VectorField, ParticleSet, JITParticle, ErrorCode, ParcelsRandom

sys.path.append('/home/jvalenti/MOAD/analysis-jose/notebooks/parcels')
from Kernels_sink import DeleteParticle, Buoyancy, AdvectionRK4_3D
from OP_functions import *

In [3]:
# Define paths
local = 0 #Set to 0 when working on server
paths = path(local)

In [4]:
Dat=xr.open_dataset(get_WW3_path(datetime(2018, 12, 23)))
path_NEMO = make_prefix(datetime(2018, 12, 23), paths['NEMO'])
Dat0=xr.open_dataset(path_NEMO + '_grid_W.nc')
Dat=xr.open_dataset(path_NEMO + '_grid_T.nc')
batt=xr.open_dataset(paths['mask'],decode_times=False)
coord=xr.open_dataset(paths['coords'],decode_times=False)
WW3 = xr.open_dataset(get_WW3_path(datetime(2018, 12, 23)))

In [10]:
N0=100
bat=batt.mbathy[0,:,:]
lat=coord.gphif[0,:,:]
lon=coord.glamf[0,:,:]

Size=bat.shape[0]*bat.shape[1]
Ny0=bat.shape[0]/Size
Nx0=bat.shape[1]/Size

rNy=Ny0/(Ny0+Nx0)
rNx=Nx0/(Ny0+Nx0)

ncP=int(Size/N0)
ncPy0=np.sqrt(ncP*(rNy/rNx))
ncPx0=ncP/ncPy0

ncPy = round(ncPy0,1)
ncPx = round(ncPx0,1)

N=int((bat.shape[0]/ncPy) * (bat.shape[1]/ncPx))


lats=[]
lons=[]
bat.shape[1]/ncPx
# for n in range(N):
#     ny=int(n*ncPy)
#     nx=int(n*ncPx)
#     lats.append(lat[ny,nx])
#     lons.append(lon[ny,nx])

398

In [17]:
lat0 = coord.gphif[0,:,:]
lon0 = coord.glamf[0,:,:]
bat = batt.mbathy[0,:,:]

Lat,Lon = [],[]
lat = np.zeros(bat.shape[1])
lon = np.zeros(bat.shape[1])
for i in range(bat.shape[0]):
    Lat,Lon = [],[]
    lat = np.append(lat,Lat,0)
    lon = np.append(lon,Lon,0)
    for j in range(bat.shape[1]):
        if bat[i,j]>1e-5:
            Lat.append(lat0[i,j])
            Lon.append(lon0[i,j])
            



dy=10
dx=10
yi=np.arange(0,lat.shape[0],dy)
xi=np.arange(0,lon.shape[1],dx)


KeyboardInterrupt: 

## Definitions

In [60]:
start = datetime(2019, 12, 10) #Start date
# Set Time length [days] and timestep [seconds]
length = 10
duration = timedelta(days=length)
dt = 90 #toggle between - or + to pick backwards or forwards
N = 6 # number of deploying locations
n = 100 # 1000   # number of particles per location
dmin = [0,0,0,0,0,70] #minimum depth
dd = 5 #max depth difference from dmin
x_offset, y_offset, zvals = p_deploy(N,n,dmin,dd)

In [61]:
from parcels import Variable

class MPParticle(JITParticle):        
    ro = Variable('ro', initial = 1370)           
    diameter = Variable('diameter', initial = 1.6e-5)
    length = Variable('length', initial = 61e-5)
    sediment = Variable('sediment', initial = 0)


In [62]:
Dat=xr.open_dataset(paths['coords'],decode_times=False)
outf_lat=Dat['nav_lat'][445,304]
outf_lon=Dat['nav_lon'][445,304]

In [63]:
lon = np.zeros([N,n])
lat = np.zeros([N,n])
# Execute run
clon, clat = [-123.901172,-125.155849,-123.207648,-122.427508,-123.399769,float(outf_lon)], [49.186308,49.975326,49.305448,47.622403,48.399420,float(outf_lat)]  # choose horizontal centre of the particle cloud
for i in range(N):
    lon[i,:]=(clon[i] + x_offset[i,:])
    lat[i,:]=(clat[i] + y_offset[i,:])
z = zvals

In [64]:
#Set start date time and the name of the output file
name = 'Sinks3' #name output file
daterange = [start+timedelta(days=i) for i in range(length)]
fn =  name + '_'.join(d.strftime('%Y%m%d')+'_1n' for d in [start, start+duration]) + '.nc'
outfile = os.path.join(paths['out'], fn)
print(outfile)

/home/jvalenti/MOAD/analysis-jose/notebooks/results/Sinks320191210_1n_20191220_1n.nc


## Simulation

In [65]:
#Fill in the list of variables that you want to use as fields
varlist=['U','V','W','R']
filenames,variables,dimensions=filename_set(start,length,varlist,local)
field_set=FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True)

filenames,variables,dimensions=filename_set(start,length,['Bathy'],local)
Bth = Field.from_netcdf(filenames['Bathy'], variables['Bathy'], dimensions,allow_time_extrapolation=True)
field_set.add_field(Bth)

         It will be opened with no decoding. Filling values might be wrongly parsed.


In [66]:
# #Load Salish output as fields
#field_set = FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True)
pset = ParticleSet.from_list(field_set, MPParticle, lon=lon, lat=lat, depth=z, time=start+timedelta(hours=2))

p_new = pset.Kernel(Buoyancy) #uncomment if you want to use additional kernels, also change the value of pset.Kernel to "AdvectionRK4_3D + p_flt"
##pset.computeTimeChunk(allow_time_extrapolation=1)
pset.execute(AdvectionRK4_3D + p_new, 
             runtime=duration, 
             dt=dt,
             output_file=pset.ParticleFile(name=outfile, outputdt=timedelta(hours=1)),
             recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})

INFO: Compiled ArrayMPParticleAdvectionRK4_3DBuoyancy ==> /tmp/parcels-2894/libf1c814e86967e80fc72ddc356ed19243_0.so


KeyboardInterrupt: 